In [1]:
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
n = 3

# mod vers.
# orig. paper: version = 1 (ResNet v1)
# Improved ResNet: version = 2 (ResNet v2)
version = 1

In [4]:
# Depth computed from n param
if version == 1:
    depth = 6*n + 2
elif version == 2:
    depth = 9*n + 2
    
#if version == 2:
#    model = resnet_v2(input_shape=input_shape, depth=depth)
#else
#    model = resnet_v1(input_shape=input_shape, depth=depth)

In [ ]:
# Some imports missing...

def resnet_v1(input_shape, depth, n_classes=10):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n + 2 (e.g., 20, 26, 32, ...)')
    n_filters = 16
    n_res_blocks = int((depth - 2) / 6)
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    
    # Instantiate stack of resid units
    for stack in range(3):
        for res_block in range(n_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2 # downsample
            y = resnet_layer(
                inputs=x, n_filters=n_filters, strides=strides)
            y = resnet_layer(
                inputs=y, n_filters=n_filters, activation=None)
            if stack > 0 and res_block == 0:
                # linear projection resid short connection to match dims
                x = resnet_layer(inputs=x, 
                                 n_filters=n_filters, 
                                 kernel_size=1, 
                                 strides=strides, 
                                 activation=None, 
                                 batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        n_filters *= 2
        
    # Add classifiers on top
    # V1 does not use batch normalization after last shortcut connection
    # ReLU
    x = AveragePooling2D(pool_size=8)
    outputs = Dense(
        n_classes, activation='softmax', kernel_initializer='he_normal'
    )(y)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model